In [265]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import pymysql
from itertools import chain
from collections import defaultdict
import numpy as np

In [30]:
conn = pymysql.connect(host = 'localhost',
                       port=3306,
                       user = 'root',
                       password = 'rootpass',
                       db = 'jeju')

In [31]:
sql_input = "SELECT * FROM jeju_data_web where contentscd = '음식점'"
sql_df = pd.read_sql_query(sql_input, conn)

C:\Users\TJ\.conda\envs\py38\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


In [32]:
sql_df

,id,name,x,y,fullRoadAddress,phone,description,keywords,imageURL,categories,bizhourInfo,menus,menuImages,reviewCount,contentscd
0,19873758,네거리식당,126.559299,33.248495,제주특별자치도 서귀포시 서문로29번길 20,064-762-5513,"성게미역국, 갈치국 등 싱싱한 재료를 이용한 제주음식을 만날 수 있는 곳입니다.","갈치국, 자리물회, 한치물회, 옥돔국, 갈치구이",https://ldb-phinf.pstatic.net/20210722_123/162...,"한식,해물,생선요리",평일 07:00~21:40 마지막 주문 20시40분,"갈치국 (소) : 15,000원/ 갈치국 (특) : 20,000원/ 성게미역국 : ...",https://ldb-phinf.pstatic.net/20171031_210/150...,1625,음식점
1,19873447,맛나식당,126.916061,33.448598,제주특별자치도 서귀포시 성산읍 동류암로 41,064-782-4771,맛나갈비 식당에는 칼칼한 생선조림은 있지만 갈비는 없습니다.오래전 가족들과 함께 이...,"갈치조림, 고등어조림",https://ldb-phinf.pstatic.net/20150831_75/1441...,"음식점,한식","매일 08:30~14:00 연중무휴, 재료 소진 시 까지 / 일요일, 수요일 휴무","갈치조림 : 13,000원/ 고등어조림 : 11,000원/ 공기밥추가 : 1,000...",https://ldb-phinf.pstatic.net/20190907_252/156...,1895,음식점
2,1927504039,제주순메밀막국수,126.301950,33.291255,제주특별자치도 서귀포시 안덕면 녹차분재로 60,064-792-0600,안녕하세요 100%제주 순메밀만 사용하는 제주순메밀막국수 입니다.,"갓짠들기름, 24시간영업, 산방산음식점, 제주막국수, 제주순메밀막국수",https://ldb-phinf.pstatic.net/20210409_285/161...,"한식,막국수",매일 06:00~23:00 마지막 주문 22:30,"들기름막국수 : 10,000원/ 명태회막국수 : 10,000원/ 수육 : 20,00...",https://ldb-phinf.pstatic.net/20220513_145/165...,1167,음식점
3,37060300,자매국수,126.517000,33.516822,제주특별자치도 제주시 탑동로11길 6,064-746-2222,2022년 6월 2일부터 영업시간이 오후 6시까지로 단축됩니다. 손님 여러분의 양해...,"고기국수, 비빔국수, 국수, 돔베고기",https://ldb-phinf.pstatic.net/20220531_107/165...,"한식,국수",매일 09:00~19:40 / 마지막 주문 19:40 / 브레이크타임 14:20~1...,"고기국수 : 9,000원/ 비빔국수 : 9,000원/ 돔베고기 소 : 17,000원...",https://ldb-phinf.pstatic.net/20220531_184/165...,4519,음식점
4,13434137,미영이네식당,126.249784,33.217709,제주특별자치도 서귀포시 대정읍 하모항구로 42,064-792-0077,"모슬포항앞에 위치한 고등어회, 방어회,여름에는 물회를 많이 드시러 오는 소박하지만 ...","고등어회, 방어회, 고등어, 방어",https://ldb-phinf.pstatic.net/20200513_33/1589...,"한식,생선회",매일 11:30~22:00 마지막 주문20:30 / 수요일 휴무 매주 수요일 휴무,"고등어회 대(大) + 탕 : 77,000원/ 고등어회 소(小) + 탕 : 55,00...",https://ldb-phinf.pstatic.net/20220115_215/164...,2000,음식점
5,13418405,우진해장국,126.519969,33.511539,제주특별자치도 제주시 서사로 11,064-757-3393,제주산 고사리를 갈아 넣고 푹 끓여 갈색 빛깔이 나는 제주식 해장국을 만날 수 있는...,"고사리육개장, 사골해장국, 녹두빈대떡, 육개장",https://ldb-phinf.pstatic.net/20150901_132/144...,"한식,향토음식",매일 06:00~22:00 연중무휴,"고사리육개장 : 10,000원/ 사골해장국 : 10,000원/ 몸국 : 10,000...",https://ldb-phinf.pstatic.net/20220203_289/164...,9910,음식점
6,1011125170,오는정김밥,126.567598,33.249664,제주특별자치도 서귀포시 동문동로 2,064-762-8927,안녕하세요 제주도 오는정김밥입니다. 홀에서는 드실수 없고 포장판매만 가능합니다. 전...,김밥,https://ldb-phinf.pstatic.net/20200920_283/160...,"음식점,한식",매일 10:00~20:00 브레이크타임 13:30~14:30 / 일요일 휴무,"오는정김밥 : 3,500원/ 치즈김밥 : 4,000원/ 떡갈비김밥 : 5,000원/...",https://ldb-phinf.pstatic.net/20200920_171/160...,1905,음식점
7,1000671392,랜디스도넛 제주애월점,126.311855,33.461275,제주특별자치도 제주시 애월읍 애월로 27-1,064-799-0610,,도넛,https://ldb-phinf.pstatic.net/20220413_244/164...,"카페,디저트,도넛",평일 10:00~19:00 / 주말 10:00~20:00,"Classic : 2,200원/ Deluxe : 2,500원/ Fancy : 2,9...",https://ldb-phinf.pstatic.net/20220116_137/164...,2647,음식점
8,16886040,오드랑베이커리,126.671521,33.542101,제주특별자치도 제주시 조천읍 조함해안로 552-3,064-784-5404,,베이커리,https://ldb-phinf.pstatic.net/20180305_26/1520...,"카페,디저트,베이커리",매일 07:00~22:00,"마농바게트 : 6,800원/ 카페모카 : 4,500원/ 먹물치아바타 : 5,500원...",https://ldb-phinf.pstatic.net/20180305_216/152...,2385,음식점
9,1296235294,중문수두리보말칼국수,126.425000,33.251579,제주특별자치도 서귀포시 천제연로 192 1층,064-739-1070,2013년 9월 오픈이래 광고한번 없이 정직하게 운영하고 있습니다. 직접 반죽한 톳...,"보말칼국수, 원조보말칼국수, 수두리보말칼국수",https://ldb-phinf.pstatic.net/20210915_121/163...,"한식,칼국수,만두","매일 08:00~17:00 휴무일 : 첫째. 셋째 화요일, 16시30분 마지막 주문...","수두리보말칼국수 : 10,000원/ 전복죽 : 13,000원/ 물만두 : 7,000...",https://ldb-phinf.pstatic.net/20220414_165/164...,1035,음식점


In [16]:
review_body = pd.read_csv('./review_body_preprocessed.csv')
review_stats = pd.read_csv('./review_stats_preprocessed.csv')

In [358]:
review_voted_keywords = pd.read_csv('./review_voted_keywords_preprocessed.csv')

In [371]:
review_voted_keywords 

,id,votedKeywords_음식이 맛있어요,votedKeywords_재료가 신선해요,votedKeywords_친절해요,votedKeywords_뷰가 좋아요,votedKeywords_가성비가 좋아요,votedKeywords_매장이 넓어요,votedKeywords_양이 많아요,votedKeywords_단체모임 하기 좋아요,votedKeywords_특별한 메뉴가 있어요,...,votedKeywords_과채가 신선해요,votedKeywords_즉석조리 식품이 맛있어요,votedKeywords_상품이 다양해요,votedKeywords_신상품이 많아요,votedKeywords_향신료가 강하지 않아요,votedKeywords_읽을만한 책이 많아요,votedKeywords_라이브공연이 훌륭해요,votedKeywords_포장이 정성스러워요,votedKeywords_추천을 잘해줘요,votedKeywords_설명이 자세해요
0,12883219,114.0,56.0,49.0,49.0,35.0,35.0,33.0,21.0,20.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1207652081,17.0,13.0,7.0,0.0,14.0,1.0,4.0,3.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,35269176,41.0,23.0,18.0,5.0,9.0,4.0,10.0,10.0,6.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,11710933,23.0,12.0,17.0,15.0,3.0,7.0,2.0,5.0,4.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1480037450,30.0,9.0,16.0,13.0,4.0,2.0,0.0,2.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12016,31507955,15.0,3.0,6.0,1.0,8.0,0.0,5.0,0.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
12017,1231669980,43.0,19.0,18.0,15.0,0.0,16.0,2.0,1.0,34.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
12018,1910356482,0.0,0.0,3.0,2.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
12019,1277648050,0.0,0.0,124.0,720.0,24.0,0.0,0.0,0.0,100.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [363]:
review_voted_keywords.fillna(0,inplace=True)

In [372]:
top3_review_stats=pd.read_csv('./top3_food_review_voted_keywords_preprocessed.csv')

In [373]:
top3_review_stats.fillna(0,inplace=True)

In [374]:
total_review_stats = pd.concat([review_voted_keywords,top3_review_stats],axis=0,ignore_index=True)

In [375]:
total_review_stats.drop_duplicates(subset='id',ignore_index=True,inplace=True)

In [377]:
total_review_stats[total_review_stats['id']==1000671392]

,id,votedKeywords_음식이 맛있어요,votedKeywords_재료가 신선해요,votedKeywords_친절해요,votedKeywords_뷰가 좋아요,votedKeywords_가성비가 좋아요,votedKeywords_매장이 넓어요,votedKeywords_양이 많아요,votedKeywords_단체모임 하기 좋아요,votedKeywords_특별한 메뉴가 있어요,...,votedKeywords_과채가 신선해요,votedKeywords_즉석조리 식품이 맛있어요,votedKeywords_상품이 다양해요,votedKeywords_신상품이 많아요,votedKeywords_향신료가 강하지 않아요,votedKeywords_읽을만한 책이 많아요,votedKeywords_라이브공연이 훌륭해요,votedKeywords_포장이 정성스러워요,votedKeywords_추천을 잘해줘요,votedKeywords_설명이 자세해요
12021,1000671392,0.0,0.0,225.0,417.0,58.0,0.0,0.0,0.0,572.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
food_db=pd.read_csv('../../data/naver_crawling/음식점db_final_concat_Cafe.csv')

In [20]:
food_db['categories']=food_db.loc[:,'categories'].apply(lambda x: eval(x))

In [21]:
food_db['category1'] = food_db.loc[:,'categories'].apply(lambda x: x[0])
food_db['category2'] = food_db.loc[:,'categories'].apply(lambda x: x[-1])

In [364]:
food_db.shape

(16477, 46)

In [41]:
right_join_df=food_db[['id','category1','category2','categories']]

In [378]:
search_db=pd.merge(left=total_review_stats,right=right_join_df,how='left',right_on='id',left_on='id')

In [379]:
search_db

,id,votedKeywords_음식이 맛있어요,votedKeywords_재료가 신선해요,votedKeywords_친절해요,votedKeywords_뷰가 좋아요,votedKeywords_가성비가 좋아요,votedKeywords_매장이 넓어요,votedKeywords_양이 많아요,votedKeywords_단체모임 하기 좋아요,votedKeywords_특별한 메뉴가 있어요,...,votedKeywords_신상품이 많아요,votedKeywords_향신료가 강하지 않아요,votedKeywords_읽을만한 책이 많아요,votedKeywords_라이브공연이 훌륭해요,votedKeywords_포장이 정성스러워요,votedKeywords_추천을 잘해줘요,votedKeywords_설명이 자세해요,category1,category2,categories
0,12883219,114.0,56.0,49.0,49.0,35.0,35.0,33.0,21.0,20.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,한식,생선회,"[한식, 생선회]"
1,1207652081,17.0,13.0,7.0,0.0,14.0,1.0,4.0,3.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,한식,생선회,"[한식, 생선회]"
2,35269176,41.0,23.0,18.0,5.0,9.0,4.0,10.0,10.0,6.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,한식,생선회,"[한식, 생선회]"
3,11710933,23.0,12.0,17.0,15.0,3.0,7.0,2.0,5.0,4.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,한식,생선회,"[한식, 생선회]"
4,1480037450,30.0,9.0,16.0,13.0,4.0,2.0,0.0,2.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,한식,돼지고기구이,"[한식, 돼지고기구이]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12028,1828477580,0.0,0.0,238.0,892.0,50.0,0.0,0.0,0.0,206.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,음식점,"카페,디저트","[음식점, 카페,디저트]"
12029,1273416923,0.0,0.0,262.0,733.0,45.0,0.0,0.0,0.0,308.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"카페,디저트",카페,"[카페,디저트, 카페]"
12030,1516216333,0.0,0.0,257.0,699.0,77.0,0.0,0.0,0.0,455.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"카페,디저트",베이커리,"[카페,디저트, 베이커리]"
12031,37191637,0.0,0.0,89.0,392.0,18.0,0.0,0.0,0.0,69.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"카페,디저트",카페,"[카페,디저트, 카페]"


In [380]:
id_index_df=search_db.set_index('id')

In [383]:
def get_same_category(restauarant_id):
    category_nm=id_index_df.loc[restauarant_id,'categories']
    conditions = (search_db['category1'].isin(category_nm)) |(search_db['category2'].isin(category_nm))
    result_df=search_db[conditions]
    result_df.reset_index(drop=True,inplace=True)
    return result_df

In [408]:
def get_sim_id_by_cossim(restauarant_id):
    df1=get_same_category(restauarant_id)
    print(f'{restauarant_id}같은 카테고리 식당들 조회 완료')
    df1.fillna(0,inplace=True)
    df2= df1.drop(['id','category1','category2','categories'],axis=1)
    arr=df2.to_numpy()
    #id를 통해 몇번째 index에 존재하는지 확인할 수 있게 dictionary 생성
    i2d = dict(zip(df1['id'],df1.index))
    idx = i2d.get(restauarant_id)
    #코사인유사도 행렬 생성
    cosine_sim = cosine_similarity(arr, arr)
    print('코사인유사도 행렬 생성완료')
    #idx를 통해 유사도 값 조회
    sim_scores = list(enumerate(cosine_sim[idx]))
    #코사인유사도를 기준으로 내림차순 정렬 0 번째는 본인
    sorted_scores=sorted(sim_scores,key=lambda x: x[1],reverse=True)
    #유사도 상위 3개 그리고 0.85이상인 값들의 index값만 담는다.
    index_lsts = [idx[0] for idx in sorted_scores[1:4] if idx[1] > 0.85]
    #인덱스를 통해 id 값 조회
    if index_lsts is None:
        sim_idlsts = None
    else:
        sim_idlsts=df1.iloc[index_lsts,0].values
    
    return (restauarant_id,sim_idlsts)

In [ ]:
def get_name_n_imageurl(restaurants_id):
    f

In [168]:
food_db[food_db['id'] == 19873758]['categories']

11564    [한식, 해물,생선요리]
Name: categories, dtype: object

In [395]:
td = get_same_category(16886040)

In [396]:
td.fillna(0,inplace=True)

C:\Users\TJ\AppData\Local\Temp\ipykernel_13768\1811282120.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  td.fillna(0,inplace=True)


In [398]:
td2 = td.drop(['id','category1','category2','categories'],axis=1)
arr=td2.to_numpy()

In [400]:
i2d = dict(zip(td['id'],td.index))
idx = i2d.get(16886040)
#코사인유사도 행렬 생성
cosine_sim = cosine_similarity(arr, arr)
sim_scores = list(enumerate(cosine_sim[idx]))

In [405]:
td.iloc[980,0]

1516216333

In [409]:
restaurants_lsts = []
sims_id = []
for i,v in enumerate(sql_df['id']):
    original_id,sim_id_results = get_sim_id_by_cossim(v)
    restaurants_lsts.append(original_id)
    sims_id.append(sim_id_results)
    print(f'{i}번째 종료')

19873758같은 카테고리 식당들 조회 완료


C:\Users\TJ\AppData\Local\Temp\ipykernel_13768\2893858214.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1.fillna(0,inplace=True)


코사인유사도 행렬 생성완료
0번째 종료
19873447같은 카테고리 식당들 조회 완료


C:\Users\TJ\AppData\Local\Temp\ipykernel_13768\2893858214.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1.fillna(0,inplace=True)


코사인유사도 행렬 생성완료
1번째 종료
1927504039같은 카테고리 식당들 조회 완료


...

코사인유사도 행렬 생성완료
51번째 종료
1509572684같은 카테고리 식당들 조회 완료
코사인유사도 행렬 생성완료
52번째 종료
16947222같은 카테고리 식당들 조회 완료


C:\Users\TJ\AppData\Local\Temp\ipykernel_13768\2893858214.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1.fillna(0,inplace=True)
C:\Users\TJ\AppData\Local\Temp\ipykernel_13768\2893858214.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1.fillna(0,inplace=True)


코사인유사도 행렬 생성완료
53번째 종료
37438036같은 카테고리 식당들 조회 완료
코사인유사도 행렬 생성완료
54번째 종료
13444829같은 카테고리 식당들 조회 완료


C:\Users\TJ\AppData\Local\Temp\ipykernel_13768\2893858214.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1.fillna(0,inplace=True)
C:\Users\TJ\AppData\Local\Temp\ipykernel_13768\2893858214.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1.fillna(0,inplace=True)


코사인유사도 행렬 생성완료
55번째 종료


In [411]:
result_df=pd.DataFrame({'id':restaurants_lsts,
             'sims_id':sims_id})

In [435]:
sim_rank = []


for index,id_lsts in enumerate(result_df['sims_id']):
    tmp_dict = {'id':result_df['id'][index]}
    for num,id_v in enumerate(id_lsts):
        condition = (food_db['id'] == id_v)
        tmp_df=food_db[condition]
        name = tmp_df['name'].values[0]
        imageURL = tmp_df['imageURL'].values[0]
        if imageURL is np.NaN:
            result = name 
        else:
            result = name + ',' + imageURL
        
        if num == 0:
            tmp_dict['sim_rank1']= result
        elif num == 1:
            tmp_dict['sim_rank2']= result
        elif num == 2:
            tmp_dict['sim_rank3'] = result
        
    sim_rank.append(tmp_dict)

In [438]:
final_df =pd.DataFrame(sim_rank)

In [439]:
final_df.shape

(56, 4)

In [440]:
sql_df.shape

(56, 15)

In [442]:
f_df=pd.merge(left=sql_df,right=final_df,how='inner',left_on='id',right_on='id')

In [443]:
f_df.to_csv('./리뷰키워드기반음식점추천추가.csv',index=False)